In [1]:
import pandas as pd
import os
import h5py
pd.options.display.float_format = '{:0,.0f}'.format

In [2]:
#Relative path between notebooks and goruped output directories
relative_path = '../../../outputs'

In [3]:
# Load Daysim results
trip = pd.read_csv(os.path.join(relative_path,r'daysim/') + r'_trip.tsv', sep='\t')
person = pd.read_csv(os.path.join(relative_path,r'daysim/') + r'_person.tsv', sep='\t')
household = pd.read_csv(os.path.join(relative_path,r'daysim/') + r'_household.tsv', sep='\t')

In [4]:
# Load survey results
surveyh5 = h5py.File(os.path.join(relative_path,r'../scripts/summarize/inputs/calibration/survey.h5'))

In [5]:
survey_trip = pd.DataFrame()
survey_person = pd.DataFrame()
survey_household = pd.DataFrame()
for col in surveyh5['Trip'].keys():
    survey_trip[col] = surveyh5['Trip'][col][:]
    
for col in surveyh5['Person'].keys():
    survey_person[col] = surveyh5['Person'][col][:]
    
for col in surveyh5['Household'].keys():
    survey_household[col] = surveyh5['Household'][col][:]

In [6]:
taz_district= pd.read_csv(os.path.join(relative_path,r'../scripts/summarize/inputs/county_taz.csv'))

In [7]:
taz_district.columns

Index([u'taz', u'geog_name', u'district', u'lat_1', u'lon_1'], dtype='object')

In [8]:
household_district = pd.merge(household, taz_district, left_on = 'hhtaz', right_on = 'taz')

In [9]:
household_survey_district = pd.merge(survey_household, taz_district, left_on = 'hhtaz', right_on = 'taz')

In [10]:
household_district.columns

Index([u'hhno', u'fraction_with_jobs_outside', u'hhsize', u'hhvehs', u'hhwkrs',
       u'hhftw', u'hhptw', u'hhret', u'hhoad', u'hhuni', u'hhhsc', u'hh515',
       u'hhcu5', u'hhincome', u'hownrent', u'hrestype', u'hhparcel',
       u'zone_id', u'hhtaz', u'hhexpfac', u'samptype', u'taz', u'geog_name',
       u'district', u'lat_1', u'lon_1'],
      dtype='object')

In [11]:
household_district.groupby(['hhvehs', 'district']).sum()['hhexpfac'].to_clipboard()

In [12]:
household_survey_district.groupby(['hhvehs', 'district']).sum()['hhexpfac'].to_clipboard()

In [13]:
drivers_trips = trip.loc[(trip['dorp']==1 ) & (trip['mode'] !=7)]

In [14]:
drivers_trips_survey = survey_trip.loc[(survey_trip['dorp']==1 ) & (survey_trip['mode'] !=7)]

In [23]:
household_trips = pd.merge(household_district, drivers_trips, on='hhno')
households_trips_survey = pd.merge(household_survey_district, drivers_trips_survey, on = 'hhno')

In [24]:
household_trips['weighted_distance'] = household_trips['hhexpfac']* household_trips['travdist']

In [26]:
household_trips.groupby('district').sum()['weighted_distance'].to_clipboard()


In [29]:
households_trips_survey['weighted_distance'] = households_trips_survey['hhexpfac']* households_trips_survey['travdist']
households_trips_survey.groupby('district').sum()['weighted_distance'].to_clipboard()